# SPM/DCM/PEB - Run first level

## Import

First, let's import the functions we will use from the spm package, and define a couple of
wrappers for the save/load MATLAB builtins.

In [1]:
import os
import numpy as np
from spm import (
    Array, Cell, Struct, Runtime,
    spm_select, spm_dcm_specify, spm_dcm_load, spm_dcm_fit, spm_dcm_fmri_check
)

num = np.asarray

# Save the field of a (scalar) struct in a .mat file
def save(f, x): return Runtime.call("save", f, "-struct", x)

# Load a .mat into a (scalar) struct
def load(*a): return Runtime.call("load", *a)

# Change directory in MATLAB
def cd(*a): return Runtime.call("cd", *a)

In [2]:
this_dir = start_dir = os.getcwd()
print(this_dir)

/Users/balbasty/Dropbox/Workspace/code/fil/dcm-peb-example-python/code


## Settings

In [3]:
# MRI scanner settings
TR = 3.6   # Repetition time (secs)
TE = 0.05  # Echo time (secs)

# Experiment settings
nsubjects   = 60
nregions    = 4
nconditions = 3

# Index of each condition in the DCM
# NOTE: Python uses 0-indexing
TASK, PICTURES, WORDS = 0, 1, 2

# Index of each region in the DCM
# NOTE: Python uses 0-indexing
lvF, ldF, rvF, rdF = 0, 1, 2, 3

## Specify DCMs (one per subject)

In [4]:
# A-matrix (on / off)
a = np.ones([nregions, nregions])
a[lvF, rdF] = 0
a[rdF, lvF] = 0
a[ldF, rvF] = 0
a[rvF, ldF] = 0

# B-matrix
b = np.zeros([nregions, nregions, 3])
b[:, :, TASK]     = np.zeros(nregions)  # Task
b[:, :, PICTURES] = np.eye(nregions)    # Pictures
b[:, :, WORDS]    = np.eye(nregions)    # Words

# C-matrix
c = np.zeros([nregions, nconditions])
c[:, TASK] = 1

# D-matrix (disabled)
d = np.zeros([nregions, nregions, 0])

In [5]:
for subject in range(nsubjects):

    name = f'sub-{subject+1:02d}'

    # Load SPM
    glm_dir = os.path.join(this_dir, '..', 'GLM', name)
    SPM     = load(os.path.join(glm_dir, 'SPM.mat'))
    SPM     = SPM.SPM

    # Load ROIs
    xY = Struct()
    f = [os.path.join(glm_dir, 'VOI_lvF_1.mat'),
         os.path.join(glm_dir, 'VOI_ldF_1.mat'),
         os.path.join(glm_dir, 'VOI_rvF_1.mat'),
         os.path.join(glm_dir, 'VOI_rdF_1.mat')]
    for r in range(len(f)):
        XY = load(f[r])
        xY[r] = XY.xY

        # Fix -- otherwise spm_dcm_specify crashes
        Ic = int(XY.xY.Ic.item())
        SPM.xCon[Ic].c = num([])

    if True:
        # Move to output directory
        cd(glm_dir)

        # Select whether to include each condition from the design matrix
        # (Task, Pictures, Words)
        include = np.ones([3, 1])

        # Specify. Corresponds to the series of questions in the GUI.
        s = Struct()
        s.name       = 'full'
        s.u          = include              # Conditions
        s.delays     = num([TR]*nregions)   # Slice timing for each region
        s.TE         = TE
        s.nonlinear  = False
        s.two_state  = False
        s.stochastic = False
        s.centre     = True
        s.induced    = 0
        s.a          = a
        s.b          = b
        s.c          = c
        s.d          = d
        DCM = spm_dcm_specify(SPM, xY, s)

        # Return to script directory
        cd(start_dir)

Initializing Matlab Runtime...


2025-03-24 18:23:54.923 mwpython[69976:2157579] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-24 18:23:54.923 mwpython[69976:2157579] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## Collate into a GCM file and estimate

In [ ]:
# Find all DCM files
dcms = spm_select('FPListRec', os.path.join(this_dir, '..', 'GLM'), 'DCM_full.mat')

# Prepare output directory
out_dir = os.path.join(this_dir, '..', 'analyses')
os.makedirs(out_dir, exist_ok=True)

# Check if it exists
if os.path.exists(os.path.join(out_dir, 'GCM_full.mat')):
    f = Runtime.call(
        'questdlg',
        'Overwrite existing GCM?', 'Overwrite?', 'Yes', 'No',
        {'Default': 'No', 'Interpreter': 'none'}
    )
    tf = (f == "Yes")
else:
    tf = True

# Collate & estimate
if tf:
    # Character array -> cell array
    # NOTE: In spm-python, `spm_select` already returns a cell array
    GCM = dcms

    # Filenames -> DCM structures
    GCM = spm_dcm_load(GCM)

    # Estimate DCMs (this won't effect original DCM files)
    GCM = spm_dcm_fit(GCM)

    # Save estimated GCM
    save(os.path.join(out_dir, 'GCM_full.mat'), {'GCM': GCM})

Loading DCMs...Done


FALLBACK (log once): Fallback to SW vertex for line stipple
FALLBACK (log once): Fallback to SW vertex processing, m_disable_code: 2000
FALLBACK (log once): Fallback to SW vertex processing in drawCore, m_disable_code: 2000


EM:(+): 1     F: 0.000e+00 dF predicted: 1.738e+02  actual: 8.792e+01 (1.10 sec)
EM:(+): 2     F: 8.792e+01 dF predicted: 2.409e+00  actual: 1.594e+00 (1.04 sec)
EM:(+): 3     F: 8.951e+01 dF predicted: 2.435e+00  actual: 2.249e+00 (0.99 sec)
EM:(+): 4     F: 9.176e+01 dF predicted: 3.497e+00  actual: 3.208e+00 (1.00 sec)
EM:(+): 5     F: 9.497e+01 dF predicted: 4.918e+00  actual: 4.473e+00 (1.05 sec)
EM:(+): 6     F: 9.944e+01 dF predicted: 6.624e+00  actual: 5.930e+00 (1.02 sec)
EM:(+): 7     F: 1.054e+02 dF predicted: 8.185e+00  actual: 7.146e+00 (1.06 sec)
EM:(+): 8     F: 1.125e+02 dF predicted: 8.798e+00  actual: 7.383e+00 (1.02 sec)
EM:(+): 9     F: 1.199e+02 dF predicted: 7.913e+00  actual: 6.506e+00 (1.02 sec)
EM:(+): 10     F: 1.264e+02 dF predicted: 6.697e+00  actual: 5.866e+00 (1.02 sec)
EM:(+): 11     F: 1.323e+02 dF predicted: 7.210e+00  actual: 6.562e+00 (1.00 sec)
EM:(+): 12     F: 1.388e+02 dF predicted: 9.865e+00  actual: 6.362e+00 (1.00 sec)
EM:(+): 13     F: 1.452e+

 actual: 2.033e+01 (1.04 sec)
EM:(+): 5     F: 2.002e+02 dF predicted: 2.712e+01  actual: 2.211e+01 (1.09 sec)
EM:(+): 6     F: 2.223e+02 dF predicted: 2.751e+01  actual: 2.088e+01 (1.09 sec)
EM:(+): 7     F: 2.432e+02 dF predicted: 2.702e+01  actual: 1.953e+01 (1.06 sec)
EM:(+): 8     F: 2.627e+02 dF predicted: 2.669e+01  actual: 1.972e+01 (0.98 sec)
EM:(+): 9     F: 2.824e+02 dF predicted: 2.162e+01  actual: 1.353e+01 (1.03 sec)
EM:(+): 10     F: 2.959e+02 dF predicted: 1.699e+01  actual: 1.020e+01 (1.04 sec)
EM:(+): 11     F: 3.061e+02 dF predicted: 1.265e+01  actual: -3.194e-01 (0.98 sec)
EM:(-): 12     F: 3.061e+02 dF predicted: 8.556e-01  actual: 3.282e-01 (0.96 sec)
EM:(+): 13     F: 3.065e+02 dF predicted: 3.428e-01  actual: 2.985e-01 (0.97 sec)
EM:(+): 14     F: 3.068e+02 dF predicted: 3.634e-01  actual: 3.083e-01 (0.97 sec)
EM:(+): 15     F: 3.071e+02 dF predicted: 3.772e-01  actual: 3.056e-01 (0.99 sec)
EM:(+): 16     F: 3.074e+02 dF predicted: 4.039e-01  actual: 3.230e-01 (

## Specify 28 alternative models structures

In [ ]:
#  These will be templates for the group analysis

# Define B-matrix for each family (factor: task)
# -------------------------------------------------------------------------
# Both
b_task_fam = Cell()
b_task_fam[0][:, :, 1] = np.ones(4)  # Objects
b_task_fam[0][:, :, 2] = np.ones(4)  # Words

# Words
b_task_fam[1][:, :, 1] = np.zeros(4)  # Objects
b_task_fam[1][:, :, 2] = np.ones(4)   # Words

# Objects
b_task_fam[2][:, :, 1] = np.ones(4)   # Objects
b_task_fam[2][:, :, 2] = np.zeros(4)  # Words

task_fam_names = ['Both', 'Words', 'Objects']

# Define B-matrix for each family (factor: dorsal-ventral)
# -------------------------------------------------------------------------
# Both
b_dv_fam = Cell()
b_dv_fam[0] = np.eye(4)

# Dorsal
b_dv_fam[1] = num([[0, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 1]])
# Ventral
b_dv_fam[2] = num([[1, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 0]])

b_dv_fam_names = ['Both' 'Dorsal' 'Ventral']

# Define B-matrix for each family (factor: left-right)
# -------------------------------------------------------------------------
# Both
b_lr_fam = Cell()
b_lr_fam[0] = np.eye(4)

# Left
b_lr_fam[1] = num([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

# Right
b_lr_fam[2] = num([[0, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])

b_lr_fam_names = ['Both', 'Left', 'Right']

# Make a DCM for each mixture of these factors
# -------------------------------------------------------------------------

# Load and unpack an example DCM
GCM_full = load(os.path.join(out_dir, 'GCM_full.mat'))
GCM_full = spm_dcm_load(GCM_full.GCM)
DCM_template = GCM_full[0, 0]
a = DCM_template.a
c = DCM_template.c
d = DCM_template.d
options = DCM_template.options

# Output cell array for new models
GCM_templates = Cell()

m = 0
task_family = Array()
b_dv_family = Array()
b_lr_family = Array()
for t in range(b_task_fam):
    for dv in range(b_dv_fam):
        for lr in range(b_lr_fam):

            # Prepare B-matrix
            b = np.zeros([4, 4, 3])
            b[:, :, 1:2] = b_dv_fam[dv] & b_lr_fam[lr] & b_task_fam[t]

            # Prepare model name
            name = print(
                'Task: %s, Dorsoventral: %s, Hemi: %s' %
                (task_fam_names[t], b_dv_fam_names[dv], b_lr_fam_names[lr])
            )

            # Build minimal DCM
            DCM = Struct()
            DCM.a       = a
            DCM.b       = b
            DCM.c       = c
            DCM.d       = d
            DCM.options = options
            DCM.name    = name
            GCM_templates[m] = DCM

            # Record the assignment of this model to each family
            task_family[m] = t
            b_dv_family[m] = dv
            b_lr_family[m] = lr
            m += 1


# Add a null model with no modulation
# -------------------------------------------------------------------------
b = np.zeros(4)
c = num([[1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0]])
name = 'Task: None'

DCM = Struct()
DCM.b[:, :, 1] = b
DCM.b[:, :, 2] = b
DCM.c          = c
DCM.name       = name

GCM_templates[m] = DCM

# Record the assignment of this model to each family
b_dv_family[m] = len(b_dv_fam)
b_lr_family[m] = len(b_lr_fam)
task_family[m] = len(b_task_fam)

m += 1

In [ ]:
# Save
GCM = GCM_templates
save(os.path.join(out_dir, 'GCM_templates.mat'),
     {
        'GCM': GCM,
        'task_family': task_family,
        'b_dv_family': b_dv_family,
        'b_lr_family': b_lr_family,
     }
)

## Run diagnostics

In [ ]:
GCM = load(os.path.join(out_dir, 'GCM_full.mat')).GCM
spm_dcm_fmri_check(GCM)